In [1]:
!pip install transformers datasets pandas scikit-learn

In [2]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/kaggle/input/ielts-dataset/ielts_writing_dataset (1).csv')
value_counts = df['Overall'].value_counts()
df = df[df['Overall'].isin(value_counts[value_counts > 1].index)]
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall
0,1,The bar chart below describes some changes abo...,"Between 1995 and 2010, a study was conducted r...",NaN,NaN,NaN,NaN,NaN,5.5
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5
2,1,The bar chart below describes some changes abo...,The left chart shows the population change hap...,NaN,NaN,NaN,NaN,NaN,5.0
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5
4,1,The graph below shows the number of overseas v...,Information about the thousands of visits from...,NaN,NaN,NaN,NaN,NaN,7.0


In [3]:
df = df[["Essay", "Overall"]]
df.head()

,Essay,Overall
0,"Between 1995 and 2010, a study was conducted r...",5.5
1,Poverty represents a worldwide crisis. It is t...,6.5
2,The left chart shows the population change hap...,5.0
3,Human beings are facing many challenges nowada...,5.5
4,Information about the thousands of visits from...,7.0


In [4]:
df_test = df.sample(n = 100, random_state = 42)
df_test.head()

,Essay,Overall
370,"From the chart, we can easily find four househ...",6.0
560,"In the diagram, the steps needed to obtain a d...",6.0
1008,This line chart depicts how much waste produce...,6.0
589,"Doing activities, either in a group or alone, ...",6.5
984,The pie chart on the left shows the distributi...,7.0


In [5]:
df = df[~df.index.isin(df_test.index)]
df.shape

(1334, 2)

In [6]:
dataset = Dataset.from_pandas(df, preserve_index = False)
dataset = dataset.train_test_split(test_size=0.1)

In [7]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=1)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Essay', 'Overall'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['Essay', 'Overall'],
        num_rows: 134
    })
})

In [9]:
def tokenize_function(examples):
    label = examples["Overall"]
    examples = tokenizer(examples["Essay"], truncation=True, padding="max_length", max_length=512)

    examples["label"] = float(label)
    return examples

for split in dataset:
    dataset[split] = dataset[split].map(tokenize_function, remove_columns=["Essay", "Overall"])

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

In [10]:
from kaggle_secrets import UserSecretsClient
import wandb

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: edbertw (edbertw-migrasia). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    logits = logits.flatten()
    labels = labels.flatten()

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [13]:
from transformers import TrainingArguments, Trainer, TrainerCallback
from transformers import EarlyStoppingCallback
import torch

training_args = TrainingArguments(output_dir="./fine_tuned_bert",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  learning_rate= 8e-5,
                                  per_device_train_batch_size=6,
                                  per_device_eval_batch_size=6,
                                  num_train_epochs=20,
                                  save_total_limit = 2,
                                  save_strategy = 'epoch',
                                  lr_scheduler_type = "cosine",
                                  optim = "adamw_torch",
                                  weight_decay = 0.1,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="r2",
                                  warmup_ratio=0.1,
                                  max_grad_norm = 1.0
                                  )

class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.training_history = {"train": [], "eval": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:  # Training logs
                self.training_history["train"].append(logs)
            elif "eval_loss" in logs:  # Evaluation logs
                self.training_history["eval"].append(logs)
'''
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)
'''

metrics_callback = MetricsCallback()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics_for_regression,
    callbacks=[metrics_callback]

)

trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,3.629500,0.905483,0.905483,0.745323,0.017473,0.440299
2,0.965800,1.175139,1.175139,0.933178,-0.275126,0.268657
3,0.804600,1.069307,1.069307,0.898954,-0.160290,0.253731
4,0.583400,0.533194,0.533194,0.599778,0.421439,0.470149
5,0.476200,0.992451,0.992451,0.848407,-0.076894,0.283582
6,0.388700,0.853838,0.853838,0.787966,0.073513,0.320896
7,0.274100,1.588344,1.588344,1.112126,-0.723490,0.186567
8,0.237200,1.147833,1.147833,0.888874,-0.245497,0.276119
9,0.195800,0.943347,0.943347,0.824254,-0.023613,0.313433
10,0.160000,1.090958,1.090958,0.884904,-0.183783,0.246269


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

{'eval_loss': 0.5331935882568359,
 'eval_mse': 0.5331935882568359,
 'eval_mae': 0.5997779965400696,
 'eval_r2': 0.4214392702843197,
 'eval_accuracy': 0.4701492537313433,
 'eval_runtime': 2.9024,
 'eval_samples_per_second': 46.169,
 'eval_steps_per_second': 4.135,
 'epoch': 20.0}

In [14]:
from transformers import pipeline
from tqdm import tqdm

def predict(X_test, model, tokenizer):
  y_pred = []
  global BERT_pipeline
  BERT_pipeline = pipeline(task = "text-classification",
                          model = model,
                          tokenizer = tokenizer,
                          truncation=True)
  for i in tqdm(range(len(X_test))):
    text = X_test.iloc[i]["Essay"]
    result = BERT_pipeline(text)
    answer = result[0]["score"]
    y_pred.append(answer)
  return pd.Series(y_pred)
y_pred = predict(df_test, model, tokenizer)
print(r2_score(df_test["Overall"],y_pred))

Device set to use cuda:0
100%|██████████| 100/100 [00:02<00:00, 41.53it/s]


0.24140699572207824


In [15]:
dataset_2 = Dataset.from_pandas(df_test, preserve_index = False)
dataset_2 = dataset_2.map(tokenize_function, remove_columns=["Essay", "Overall"])

trainer.eval_dataset=dataset_2
trainer.evaluate()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.8523553609848022,
 'eval_mse': 0.8523553609848022,
 'eval_mae': 0.717325747013092,
 'eval_r2': 0.24140679266151754,
 'eval_accuracy': 0.46,
 'eval_runtime': 2.1835,
 'eval_samples_per_second': 45.799,
 'eval_steps_per_second': 4.122,
 'epoch': 20.0}